In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm

In [3]:
pdbs_to_be_tested = ["1BG3", "2PGI", "4PFK", "4ALD", "3YPI", "3GPD",
                    "13PK", "1BQ3", "7ENL", "1A3W"]

In [4]:
molsys = msm.convert('1TCD')

/home/diego/projects@dprada/MolSysMT/molsysmt/item/mmtf_MMTFDecoder/to_molsysmt_Topology.py:32: UserWarning: The structure in the PDB has biological assemblies. There are geometrical transformations proposed in the structure. See the following issue in the source code repository: https://github.com/uibcdf/MolSysMT/issues/33
  warnings.warn(warning_message)


In [5]:
msm.get(molsys, element='atom', selection='atom_name=="CA"', b_factor=True)

array([0.3328, 0.3111, 0.239 , 0.2385, 0.201 , 0.186 , 0.1705, 0.1616,
       0.1298, 0.1732, 0.1408, 0.1007, 0.1967, 0.1833, 0.1565, 0.2292,
       0.2102, 0.1982, 0.2203, 0.2288, 0.2272, 0.2139, 0.2267, 0.255 ,
       0.2428, 0.2568, 0.2674, 0.3072, 0.3355, 0.3647, 0.4252, 0.492 ,
       0.3215, 0.2924, 0.2533, 0.2014, 0.1873, 0.1458, 0.1892, 0.144 ,
       0.1369, 0.1646, 0.1665, 0.1359, 0.1546, 0.1348, 0.1472, 0.1898,
       0.2226, 0.237 , 0.237 , 0.1941, 0.2349, 0.2804, 0.2679, 0.2894,
       0.2826, 0.26  , 0.2202, 0.1566, 0.1258, 0.1409, 0.1565, 0.168 ,
       0.1677, 0.1625, 0.1839, 0.2152, 0.1872, 0.1306, 0.2264, 0.1455,
       0.1369, 0.1067, 0.1309, 0.1474, 0.1163, 0.1349, 0.1979, 0.1335,
       0.1477, 0.1816, 0.1822, 0.1468, 0.1685, 0.1958, 0.2099, 0.1891,
       0.1683, 0.1915, 0.182 , 0.1941, 0.2155, 0.1969, 0.1253, 0.2355,
       0.1681, 0.2279, 0.2386, 0.2215, 0.2336, 0.2263, 0.1991, 0.2399,
       0.2115, 0.1539, 0.1867, 0.2272, 0.2446, 0.1334, 0.1734, 0.1986,
       0.214 , 0.1625, 0.2291, 0.2172, 0.1637, 0.1675, 0.1957, 0.2229,
       0.1956, 0.1967, 0.2171, 0.2037, 0.1709, 0.191 , 0.2014, 0.1818,
       0.2867, 0.2665, 0.247 , 0.2624, 0.2562, 0.2361, 0.2879, 0.2768,
       0.3008, 0.264 , 0.231 , 0.2619, 0.2277, 0.237 , 0.2483, 0.2425,
       0.2187, 0.1865, 0.2259, 0.2324, 0.2862, 0.2851, 0.2771, 0.275 ,
       0.2836, 0.3096, 0.305 , 0.3282, 0.3268, 0.2615, 0.2896, 0.2343,
       0.2356, 0.2194, 0.1804, 0.2049, 0.242 , 0.2504, 0.2155, 0.3278,
       0.4057, 0.3734, 0.4192, 0.4748, 0.4627, 0.4491, 0.452 , 0.4358,
       0.3607, 0.3558, 0.309 , 0.3033, 0.2745, 0.225 , 0.2487, 0.2534,
       0.2147, 0.2203, 0.234 , 0.2344, 0.2119, 0.2907, 0.2834, 0.214 ,
       0.2042, 0.295 , 0.3167, 0.3605, 0.3223, 0.3123, 0.3462, 0.2816,
       0.2678, 0.2674, 0.2629, 0.2085, 0.225 , 0.1907, 0.1766, 0.2443,
       0.2216, 0.252 , 0.264 , 0.2439, 0.25  , 0.2423, 0.3935, 0.3084,
       0.2693, 0.2938, 0.3378, 0.2274, 0.2314, 0.2887, 0.3084, 0.2626,
       0.2148, 0.2421, 0.2597, 0.1656, 0.1627, 0.1428, 0.1649, 0.1971,
       0.2154, 0.208 , 0.2404, 0.2069, 0.2301, 0.222 , 0.2726, 0.2295,
       0.2597, 0.282 , 0.2308, 0.2018, 0.2492, 0.2475, 0.2915, 0.3919,
       0.5225, 0.2563, 0.1908, 0.1528, 0.1543, 0.1169, 0.1211, 0.1051,
       0.1175, 0.1669, 0.1663, 0.1067, 0.1025, 0.1507, 0.1934, 0.198 ,
       0.2662, 0.2568, 0.2637, 0.2359, 0.2037, 0.1747, 0.1717, 0.1576,
       0.1715, 0.1346, 0.1515, 0.1384, 0.2038, 0.1934, 0.2581, 0.2549,
       0.3182, 0.215 , 0.1961, 0.1652, 0.1282, 0.1215, 0.1335, 0.1523,
       0.1056, 0.0926, 0.1159, 0.1275, 0.1325, 0.1203, 0.1126, 0.1235,
       0.1708, 0.0865, 0.1309, 0.1552, 0.151 , 0.1679, 0.0926, 0.1218,
       0.1906, 0.1728, 0.1502, 0.1447, 0.1224, 0.1441, 0.0874, 0.1028,
       0.1451, 0.1841, 0.1197, 0.1715, 0.1687, 0.1676, 0.1538, 0.1726,
       0.1335, 0.1504, 0.1477, 0.1224, 0.1195, 0.1488, 0.1276, 0.1927,
       0.1834, 0.1441, 0.0994, 0.1947, 0.1812, 0.1495, 0.1751, 0.1713,
       0.1377, 0.1079, 0.1596, 0.1504, 0.1282, 0.1765, 0.1159, 0.1437,
       0.1283, 0.119 , 0.1731, 0.2307, 0.2574, 0.2407, 0.1887, 0.1589,
       0.1105, 0.2137, 0.1643, 0.1666, 0.1745, 0.1608, 0.1299, 0.1164,
       0.1758, 0.1432, 0.1571, 0.2036, 0.1927, 0.1768, 0.2334, 0.1715,
       0.122 , 0.1416, 0.1484, 0.13  , 0.1726, 0.1887, 0.1222, 0.1389,
       0.1483, 0.2092, 0.2137, 0.1914, 0.251 , 0.3028, 0.2891, 0.2078,
       0.2284, 0.1669, 0.1738, 0.1981, 0.1254, 0.1568, 0.1521, 0.2002,
       0.2018, 0.1568, 0.1733, 0.1391, 0.1668, 0.1574, 0.2205, 0.151 ,
       0.1878, 0.2761, 0.2762, 0.2654, 0.2545, 0.2148, 0.168 , 0.1462,
       0.1235, 0.1403, 0.1478, 0.1364, 0.093 , 0.1256, 0.1301, 0.1278,
       0.1731, 0.1752, 0.1951, 0.1879, 0.2896, 0.2582, 0.2441, 0.4178,
       0.291 , 0.1784, 0.1769, 0.155 , 0.1412, 0.1707, 0.1222, 0.151 ,
       0.1249, 0.0908, 0.1666, 0.2027, 0.1705, 0.1889, 0.1882, 0.1912,
       0.1605, 0.2262, 0.2058, 0.2035, 0.2012, 0.2346, 0.2207, 0.2399,
       0.1914, 0.195 , 0

In [ ]:
molsys = msm.build.remove_atoms_with_alternate_locations(molsys)

In [ ]:
msm.view(molsys)

In [ ]:
my_enm = oenm.ElasticNetworkModel('1TCD', selection='atom_name=="CA"', cutoff='12 angstroms')